In [18]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [19]:
drop_tables = """
-- =========================
-- DROP TABLES
-- =========================

DROP TABLE IF EXISTS Event_Medikament CASCADE;
DROP TABLE IF EXISTS Arzt_Event CASCADE;
DROP TABLE IF EXISTS Event CASCADE;
DROP TABLE IF EXISTS Behandlung CASCADE;
DROP TABLE IF EXISTS Medikament CASCADE;
DROP TABLE IF EXISTS Bett CASCADE;
DROP TABLE IF EXISTS Reserve CASCADE;
DROP TABLE IF EXISTS Pflegekraft CASCADE;
DROP TABLE IF EXISTS Arzt CASCADE;
DROP TABLE IF EXISTS Patient CASCADE;
DROP TABLE IF EXISTS Raum CASCADE;
DROP TABLE IF EXISTS Station CASCADE;
DROP TABLE IF EXISTS Krankenhaus CASCADE;
"""



In [20]:
conn.rollback()

cur.execute(drop_tables)
conn.commit()



In [21]:
create_tables = """
CREATE TABLE Krankenhaus (
    KH_ID INT PRIMARY KEY,
    Strasse VARCHAR(100) NOT NULL,
    Hausnummer VARCHAR(10) NOT NULL,
    Postleitzahl VARCHAR(10) NOT NULL,
    Stadt VARCHAR(50) NOT NULL,
    Name VARCHAR(100) NOT NULL,
    CHECK (Postleitzahl ~ '^[0-9]{4}$')
);

CREATE TABLE Station (
    Station_ID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL,
    Kapazitaet INT NOT NULL,
    KH_ID INT NOT NULL,
    CHECK (Kapazitaet > 0),
    FOREIGN KEY (KH_ID) REFERENCES Krankenhaus(KH_ID)
);

CREATE TABLE Raum (
    Raumnummer INT PRIMARY KEY,
    Station_ID INT NOT NULL,
    Art VARCHAR(50),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Pflegekraft (
    Pflege_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Station_ID INT NOT NULL,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Patient (
    Patient_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Versicherungsnummer VARCHAR(30) UNIQUE NOT NULL,
    Geburtsdatum DATE NOT NULL,
    Strasse VARCHAR(100),
    Hausnummer VARCHAR(10),
    Postleitzahl VARCHAR(10),
    Stadt VARCHAR(50),
    Elektronisches_Patientendossier_ID VARCHAR(50) UNIQUE,
    Station_ID INT,
    CHECK (Postleitzahl IS NULL OR Postleitzahl ~ '^[0-9]{4}$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Bett (
    Bett_ID INT PRIMARY KEY,
    Status VARCHAR(20) NOT NULL,
    Patient_ID INT,
    Raum_ID INT NOT NULL,
    CHECK (Status IN ('frei', 'belegt')),
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    FOREIGN KEY (Raum_ID) REFERENCES Raum(Raumnummer)
);

CREATE TABLE Arzt (
    Arzt_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    ist_extern BOOLEAN NOT NULL,
    Station_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Reserve (
    Reserve_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Arzt_ID INT,
    Station_ID INT,
    Pflegekraft_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID),
    FOREIGN KEY (Pflegekraft_ID) REFERENCES Pflegekraft(Pflege_ID)
);

CREATE TABLE Behandlung (
    Behandlungs_ID INT PRIMARY KEY,
    abgeschlossen BOOLEAN NOT NULL,
    Beschreibung VARCHAR(255),
    Start_Datum DATE NOT NULL,
    End_Datum DATE,
    Patient_ID INT NOT NULL,
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    CHECK (End_Datum IS NULL OR End_Datum >= Start_Datum)
);

CREATE TABLE Medikament (
    Medikament_ID INT PRIMARY KEY,
    Wirkstoff VARCHAR(100),
    Name VARCHAR(100) NOT NULL   
);

CREATE TABLE Event (
    Event_ID INT PRIMARY KEY,
    Beschreibung VARCHAR(255),
    Uhrzeit TIME NOT NULL,
    Datum DATE NOT NULL,
    RaumID INT NOT NULL,
    Behandlung_ID INT NOT NULL,
    FOREIGN KEY (RaumID) REFERENCES Raum(Raumnummer),
    FOREIGN KEY (Behandlung_ID) REFERENCES Behandlung(Behandlungs_ID)
);


CREATE TABLE Arzt_Event (
    Arzt_ID INT NOT NULL,
    Event_ID INT NOT NULL,
    PRIMARY KEY (Arzt_ID, Event_ID),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID)
);

CREATE TABLE Event_Medikament (
    Event_ID INT NOT NULL,
    Medikament_ID INT NOT NULL,
    PRIMARY KEY (Event_ID, Medikament_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Medikament_ID) REFERENCES Medikament(Medikament_ID)
);
"""
cur.execute(create_tables)
conn.commit()


In [22]:
import random
import datetime
from faker import Faker

fake = Faker("de_CH")
random.seed(1)

TODAY = datetime.date(2025, 12, 12)

def phone(i: int) -> str:
    return f"+41 79 {i:06d}"

# =========================
# HARDCODED DATA
# =========================

BEHANDLUNG_CASES = [
    "Appendektomie Nachkontrolle",
    "Blutdruckeinstellung",
    "Diabetes Verlaufskontrolle",
    "Atemwegsinfekt Behandlung",
    "Herz-Kreislauf Check",
    "Wundversorgung",
    "Schmerztherapie",
    "Magen-Darm Untersuchung",
    "Präoperative Abklärung",
    "Postoperative Nachsorge"
]

EVENT_BESCHREIBUNGEN = [
    "Visite",
    "Operation",
    "Nachkontrolle",
    "Medikamentengabe",
    "Notfallbehandlung",
    "Diagnostische Untersuchung",
    "Pflegevisite",
    "Therapieanpassung",
    "Laborbesprechung",
    "Entlassungsgespräch"
]

STATION_NAMES = [
    "Chirurgie",
    "Innere Medizin",
    "Pädiatrie",
    "Notfall",
    "Intensivstation",
    "Radiologie"
]

RAUM_ARTEN = [
    "Operationssaal",
    "Untersuchungsraum",
    "Patientenzimmer",
    "Aufenthaltsraum"
]

MEDIKAMENTE = [
    (1, "Paracetamol", "Paracetamol"),
    (2, "Ibuprofen", "Ibuprofen"),
    (3, "Aspirin", "Acetylsalicylsäure"),
    (4, "Amoxicillin", "Amoxicillin"),
    (5, "Insulin", "Insulin")
]

# =========================
# 1) Krankenhaus (3)
# =========================
for kh_id in range(1, 4):
    cur.execute("""
        INSERT INTO Krankenhaus (KH_ID, Strasse, Hausnummer, Postleitzahl, Stadt, Name)
        VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT (KH_ID) DO NOTHING
    """, (
        kh_id,
        fake.street_name(),
        fake.building_number(),
        f"{1000 + kh_id:04d}",
        fake.city(),
        f"Kantonsspital {kh_id}"
    ))
conn.commit()

# =========================
# 2) Stationen (mind. 5 pro Krankenhaus, hier: 6 pro KH => 18)
# =========================
station_ids = []
station_name_by_id = {}

sid = 1
for kh_id in range(1, 4):
    for name in STATION_NAMES:  # 6 pro KH
        kap = random.randint(30, 65)
        cur.execute("""
            INSERT INTO Station (Station_ID, Name, Kapazitaet, KH_ID)
            VALUES (%s,%s,%s,%s)
            ON CONFLICT (Station_ID) DO NOTHING
        """, (sid, name, kap, kh_id))
        station_ids.append(sid)
        station_name_by_id[sid] = name
        sid += 1
conn.commit()

NUM_STATIONS = len(station_ids)

# Gewichte für realistische Verteilung
def station_weight(station_id: int) -> int:
    n = station_name_by_id.get(station_id, "")
    if n == "Notfall":
        return 7
    if n == "Innere Medizin":
        return 6
    if n == "Chirurgie":
        return 5
    if n == "Intensivstation":
        return 4
    if n == "Pädiatrie":
        return 3
    if n == "Radiologie":
        return 2
    return 2

station_weights = [station_weight(s) for s in station_ids]

# =========================
# 3) Räume (pro Station mehrere, Betten nur in Patientenzimmer/Aufenthaltsraum)
# =========================
raum_id = 1
rooms_by_station = {s: [] for s in station_ids}     # alle Räume
sleep_rooms_by_station = {s: [] for s in station_ids}  # nur Patientenzimmer/Aufenthaltsraum
non_op_rooms_by_station = {s: [] for s in station_ids} # alles außer Operationssaal

for s in station_ids:
    # pro Station: 1 OP, 2 Untersuchung, 6 Patientenzimmer, 1 Aufenthaltsraum => 10 Räume
    layout = [
        ("Operationssaal", 1),
        ("Untersuchungsraum", 2),
        ("Patientenzimmer", 6),
        ("Aufenthaltsraum", 1),
    ]
    for art, count in layout:
        for _ in range(count):
            cur.execute("""
                INSERT INTO Raum (Raumnummer, Station_ID, Art)
                VALUES (%s,%s,%s)
                ON CONFLICT (Raumnummer) DO NOTHING
            """, (raum_id, s, art))

            rooms_by_station[s].append(raum_id)
            if art in ("Patientenzimmer", "Aufenthaltsraum"):
                sleep_rooms_by_station[s].append(raum_id)
            if art != "Operationssaal":
                non_op_rooms_by_station[s].append(raum_id)

            raum_id += 1

conn.commit()
NUM_RAEUME = raum_id - 1

# =========================
# 4) Medikamente (5 hardcoded)
# =========================
for mid, name, wirkstoff in MEDIKAMENTE:
    cur.execute("""
        INSERT INTO Medikament (Medikament_ID, Wirkstoff, Name)
        VALUES (%s,%s,%s)
        ON CONFLICT (Medikament_ID) DO NOTHING
    """, (mid, wirkstoff, name))
conn.commit()

# =========================
# 5) Patienten (~150), verteilt über Stationen
# =========================
NUM_PATIENTS = 150
patients_by_station = {s: [] for s in station_ids}

for pid in range(1, NUM_PATIENTS + 1):
    s = random.choices(station_ids, weights=station_weights, k=1)[0]
    patients_by_station[s].append(pid)

    cur.execute("""
        INSERT INTO Patient
        (Patient_ID, Vorname, Nachname, Versicherungsnummer, Geburtsdatum,
         Strasse, Hausnummer, Postleitzahl, Stadt, Elektronisches_Patientendossier_ID, Station_ID)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (Patient_ID) DO NOTHING
    """, (
        pid,
        fake.first_name(),
        fake.last_name(),
        f"CH{pid:08d}",
        fake.date_of_birth(minimum_age=0, maximum_age=95),
        fake.street_name(),
        fake.building_number(),
        f"{1000 + (pid % 50):04d}",
        fake.city(),
        f"EPD-{pid}",
        s
    ))
conn.commit()

# =========================
# 6) Ärzte (mind. 1 pro Station, insgesamt ~26)
# =========================
doctor_ids_by_station = {s: [] for s in station_ids}
doctor_id = 1

# mindestens 1 pro Station
for s in station_ids:
    cur.execute("""
        INSERT INTO Arzt (Arzt_ID, Telefonnummer, Vorname, Nachname, ist_extern, Station_ID)
        VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT (Arzt_ID) DO NOTHING
    """, (
        doctor_id,
        phone(doctor_id),
        fake.first_name(),
        fake.last_name(),
        (doctor_id % 7 == 0),
        s
    ))
    doctor_ids_by_station[s].append(doctor_id)
    doctor_id += 1

# ein paar extra Ärzte (damit manche Stationen mehr haben)
EXTRA_DOCTORS = 8  # 18 + 8 = 26
for _ in range(EXTRA_DOCTORS):
    s = random.choices(station_ids, weights=station_weights, k=1)[0]
    cur.execute("""
        INSERT INTO Arzt (Arzt_ID, Telefonnummer, Vorname, Nachname, ist_extern, Station_ID)
        VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT (Arzt_ID) DO NOTHING
    """, (
        doctor_id,
        phone(doctor_id),
        fake.first_name(),
        fake.last_name(),
        (doctor_id % 7 == 0),
        s
    ))
    doctor_ids_by_station[s].append(doctor_id)
    doctor_id += 1

conn.commit()
NUM_AERZTE = doctor_id - 1

# =========================
# 7) Pflegekräfte (mind. 2 pro Station, insgesamt ~55)
# =========================
pflege_ids_by_station = {s: [] for s in station_ids}
pflege_id = 1

# mindestens 2 pro Station
for s in station_ids:
    for _ in range(2):
        cur.execute("""
            INSERT INTO Pflegekraft (Pflege_ID, Telefonnummer, Vorname, Nachname, Station_ID)
            VALUES (%s,%s,%s,%s,%s)
            ON CONFLICT (Pflege_ID) DO NOTHING
        """, (
            pflege_id,
            phone(2000 + pflege_id),
            fake.first_name(),
            fake.last_name(),
            s
        ))
        pflege_ids_by_station[s].append(pflege_id)
        pflege_id += 1

# extras ungleich verteilt
EXTRA_PFLEGE = 19  # 36 + 19 = 55
for _ in range(EXTRA_PFLEGE):
    s = random.choices(station_ids, weights=station_weights, k=1)[0]
    cur.execute("""
        INSERT INTO Pflegekraft (Pflege_ID, Telefonnummer, Vorname, Nachname, Station_ID)
        VALUES (%s,%s,%s,%s,%s)
        ON CONFLICT (Pflege_ID) DO NOTHING
    """, (
        pflege_id,
        phone(2000 + pflege_id),
        fake.first_name(),
        fake.last_name(),
        s
    ))
    pflege_ids_by_station[s].append(pflege_id)
    pflege_id += 1

conn.commit()
NUM_PFLEGE = pflege_id - 1

# =========================
# 8) Reserve (15)
# =========================
NUM_RESERVE = 15
for rid in range(1, NUM_RESERVE + 1):
    s = random.choice(station_ids)
    arzt = random.choice(doctor_ids_by_station[s]) if random.random() < 0.75 else None
    pk = random.choice(pflege_ids_by_station[s]) if random.random() < 0.85 else None

    cur.execute("""
        INSERT INTO Reserve (Reserve_ID, Telefonnummer, Arzt_ID, Station_ID, Pflegekraft_ID)
        VALUES (%s,%s,%s,%s,%s)
        ON CONFLICT (Reserve_ID) DO NOTHING
    """, (
        rid,
        phone(5000 + rid),
        arzt,
        s,
        pk
    ))
conn.commit()

# =========================
# 9) Betten (nur in Patientenzimmer/Aufenthaltsraum, und Patienten passend zur Station)
# Bett Schema: (Bett_ID, Status, Patient_ID, Raum_ID)
# =========================
bett_id = 1

# Ziel: pro Station ~25 Betten (realistisch), Belegung ~70%
for s in station_ids:
    sleep_rooms = sleep_rooms_by_station[s]
    if not sleep_rooms:
        continue

    station_patients = patients_by_station[s].copy()
    random.shuffle(station_patients)

    num_beds = 25
    occupy_target = int(num_beds * 0.7)

    for i in range(num_beds):
        raum = random.choice(sleep_rooms)

        if i < occupy_target and station_patients:
            pid = station_patients.pop()
            status = "belegt"
        else:
            pid = None
            status = "frei"

        cur.execute("""
            INSERT INTO Bett (Bett_ID, Status, Patient_ID, Raum_ID)
            VALUES (%s,%s,%s,%s)
            ON CONFLICT (Bett_ID) DO NOTHING
        """, (bett_id, status, pid, raum))
        bett_id += 1

conn.commit()
NUM_BETTEN = bett_id - 1

# =========================
# 10) Behandlungen (~220), ungleich verteilt, aber sinnvoll
# Regeln:
#  - abgeschlossen=False: Start nach TODAY, End NULL
#  - abgeschlossen=True: Start >= 2022-01-01, End <= TODAY, End>=Start
# =========================
NUM_BEHANDLUNGEN = 220

# einige "heavy patients" für ungleiche Verteilung
all_patients = list(range(1, NUM_PATIENTS + 1))
heavy_patients = set(random.sample(all_patients, 25))
patient_weights = [5 if p in heavy_patients else 1 for p in all_patients]

# pro Patient mindestens 1 Behandlung
behandlung_id = 1
treatments_by_patient = {p: [] for p in all_patients}

def create_treatment(bid: int, patient_id: int):
    abgeschlossen = (random.random() > 0.30)

    if abgeschlossen:
        start = fake.date_between(datetime.date(2022, 1, 1), TODAY)
        end = start + datetime.timedelta(days=random.randint(1, 30))
        if end > TODAY:
            end = TODAY
    else:
        start = TODAY + datetime.timedelta(days=random.randint(1, 90))
        end = None

    cur.execute("""
        INSERT INTO Behandlung
        (Behandlungs_ID, abgeschlossen, Beschreibung, Start_Datum, End_Datum, Patient_ID)
        VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT (Behandlungs_ID) DO NOTHING
    """, (
        bid,
        abgeschlossen,
        random.choice(BEHANDLUNG_CASES),
        start,
        end,
        patient_id
    ))

    treatments_by_patient[patient_id].append((bid, abgeschlossen, start, end))

for p in all_patients:
    create_treatment(behandlung_id, p)
    behandlung_id += 1

# restliche Behandlungen auffüllen (ungleiche Verteilung)
while behandlung_id <= NUM_BEHANDLUNGEN:
    p = random.choices(all_patients, weights=patient_weights, k=1)[0]
    create_treatment(behandlung_id, p)
    behandlung_id += 1

conn.commit()

# =========================
# 11) Events
# Ziel:
#  - viele Events in den nächsten 90 Tagen
#  - zusätzlich: pro Arzt am TODAY mind. 3 Events
#  - Arzt bekommt nur Events für Patienten auf seiner Station
# =========================
# Patient -> Station (für saubere Zuordnung)
cur.execute("SELECT patient_id, station_id FROM patient;")
patient_station = {pid: sid for pid, sid in cur.fetchall()}

# Behandlung -> Patient
cur.execute("SELECT behandlungs_id, patient_id, abgeschlossen, start_datum, end_datum FROM behandlung;")
behandlungs_rows = cur.fetchall()

# Für TODAY-Events brauchen wir je Station genug Behandlungen mit Patienten dieser Station
treatments_by_station = {s: [] for s in station_ids}
for bid, pid, abgeschlossen, start_d, end_d in behandlungs_rows:
    s = patient_station.get(pid)
    if s is None:
        continue
    treatments_by_station[s].append((bid, pid, abgeschlossen, start_d, end_d))

event_id = 1

def insert_event(eid: int, beschr: str, uhrzeit, datum: datetime.date, raum_id: int, behandlung_id: int):
    cur.execute("""
        INSERT INTO Event (Event_ID, Beschreibung, Uhrzeit, Datum, RaumID, Behandlung_ID)
        VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT (Event_ID) DO NOTHING
    """, (eid, beschr, uhrzeit, datum, raum_id, behandlung_id))

def insert_arzt_event(arzt_id: int, event_id: int):
    cur.execute("""
        INSERT INTO Arzt_Event (Arzt_ID, Event_ID)
        VALUES (%s,%s)
        ON CONFLICT (Arzt_ID, Event_ID) DO NOTHING
    """, (arzt_id, event_id))

def insert_event_medikament(event_id: int):
    meds = random.sample([1, 2, 3, 4, 5], random.randint(1, 2))
    for mid in meds:
        cur.execute("""
            INSERT INTO Event_Medikament (Event_ID, Medikament_ID)
            VALUES (%s,%s)
            ON CONFLICT (Event_ID, Medikament_ID) DO NOTHING
        """, (event_id, mid))

# 11a) Basis-Events pro Behandlung (1-3), mit Schwerpunkt rund um TODAY (±7 Tage) bei abgeschlossenen
for bid, pid, abgeschlossen, start_d, end_d in behandlungs_rows:
    s = patient_station.get(pid)
    if s is None:
        continue

    # Räume: für Events lieber NICHT im Operationssaal, außer wenn Beschreibung "Operation"
    non_op_rooms = non_op_rooms_by_station[s]
    all_rooms = rooms_by_station[s]

    n = random.randint(1, 3)

    for _ in range(n):
        beschr = random.choice(EVENT_BESCHREIBUNGEN)

        if abgeschlossen:
            # Schwerpunkt: rund um TODAY, aber innerhalb Start..End (End fallback TODAY)
            end_eff = end_d if end_d is not None else TODAY
            if end_eff < start_d:
                end_eff = start_d

            # Wunsch: viel in der Woche rund um TODAY
            if random.random() < 0.65:
                window_start = max(start_d, TODAY - datetime.timedelta(days=7))
                window_end = min(end_eff, TODAY + datetime.timedelta(days=7))
                if window_end < window_start:
                    window_start = start_d
                    window_end = end_eff
                ev_date = fake.date_between(window_start, window_end)
            else:
                ev_date = fake.date_between(start_d, end_eff)
        else:
            # offene Behandlung: nur Zukunft
            ev_date = TODAY + datetime.timedelta(days=random.randint(1, 90))

        # Raumwahl
        if beschr == "Operation":
            raum = random.choice(all_rooms)  # darf OP sein
        else:
            raum = random.choice(non_op_rooms) if non_op_rooms else random.choice(all_rooms)

        insert_event(event_id, beschr, fake.time(), ev_date, raum, bid)
        # Arzt_Event wird später sauber zugeordnet (nach Station)
        event_id += 1

conn.commit()

# 11b) Arzt_Event sauber zuordnen: Event -> Behandlung -> Patient -> Station -> Arzt aus gleicher Station
cur.execute("""
    SELECT e.event_id, b.patient_id
    FROM event e
    JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
""")
event_patient = cur.fetchall()

for eid, pid in event_patient:
    s = patient_station.get(pid)
    if s is None:
        continue
    doctors = doctor_ids_by_station.get(s, [])
    if not doctors:
        continue
    arzt = random.choice(doctors)
    insert_arzt_event(arzt, eid)

conn.commit()

# 11c) Garantie: jeder Arzt hat am TODAY mindestens 3 Events (und nur auf eigener Station)
cur.execute("SELECT arzt_id, station_id FROM arzt;")
arzt_station = {aid: sid for aid, sid in cur.fetchall()}

cur.execute("""
    SELECT ae.arzt_id, COUNT(*)
    FROM arzt_event ae
    JOIN event e ON e.event_id = ae.event_id
    WHERE e.datum = %s
    GROUP BY ae.arzt_id
""", (TODAY,))
counts_today = {aid: c for aid, c in cur.fetchall()}

MIN_TODAY_EVENTS = 3

for arzt_id, s in arzt_station.items():
    have = counts_today.get(arzt_id, 0)
    need = max(0, MIN_TODAY_EVENTS - have)
    if need == 0:
        continue

    # passende Behandlungen: Patienten auf derselben Station (egal ob abgeschlossen oder nicht, aber TODAY Event soll sinnvoll sein)
    # Wir nehmen bevorzugt abgeschlossene oder solche die TODAY zulassen. Am einfachsten: abgeschlossene.
    candidates = [t for t in treatments_by_station[s] if t[2] is True]  # abgeschlossen==True
    if not candidates:
        # Fallback: irgendeine Behandlung der Station
        candidates = treatments_by_station[s]

    non_op_rooms = non_op_rooms_by_station[s]
    if not non_op_rooms:
        non_op_rooms = rooms_by_station[s]

    for _ in range(need):
        bid, pid, abgeschlossen, start_d, end_d = random.choice(candidates)
        insert_event(event_id, "Visite", fake.time(), TODAY, random.choice(non_op_rooms), bid)
        insert_arzt_event(arzt_id, event_id)
        event_id += 1

conn.commit()

# 12) Event_Medikament für ALLE Events (inkl. Garantie-Events)
cur.execute("SELECT event_id FROM event;")
all_event_ids = [r[0] for r in cur.fetchall()]

for eid in all_event_ids:
    insert_event_medikament(eid)

conn.commit()

print("✔ Befüllung fertig (konsistent):")
print(f"Krankenhäuser: 3")
print(f"Stationen: {NUM_STATIONS} | Räume: {NUM_RAEUME} | Betten: {NUM_BETTEN}")
print(f"Patienten: {NUM_PATIENTS} | Ärzte: {NUM_AERZTE} | Pflegekräfte: {NUM_PFLEGE} | Reserve: {NUM_RESERVE}")
print(f"Behandlungen: {NUM_BEHANDLUNGEN} | Events: {len(all_event_ids)}")
print(f"TODAY: {TODAY} (jeder Arzt hat >= {MIN_TODAY_EVENTS} Events am TODAY)")


✔ Befüllung fertig (konsistent):
Krankenhäuser: 3
Stationen: 18 | Räume: 180 | Betten: 450
Patienten: 150 | Ärzte: 26 | Pflegekräfte: 55 | Reserve: 15
Behandlungen: 220 | Events: 509
TODAY: 2025-12-12 (jeder Arzt hat >= 3 Events am TODAY)


In [23]:
conn.rollback()
